## DSCC202-402 Data Science at Scale Final Project
### Tracking Tweet sentiment at scale using a pretrained transformer (classifier)
<p>Consider the following illustration of the end to end system that you will be building.  Each student should do their own work.  The project will demonstrate your understanding of Spark Streaming, the medalion data architecture using Delta Lake, Spark Inference at Scale using an MLflow packaged model as well as Exploritory Data Analysis and System Tracking and Monitoring.</p>
<br><br>
<img src="https://data-science-at-scale.s3.amazonaws.com/images/pipeline.drawio.png">

<p>
You will be pulling an updated copy of the course GitHub repositiory: <a href="https://github.com/lpalum/dscc202-402-spring2024">The Repo</a>.  If you are unclear on how to pull an updated copy using the GitHub command line, the following <a href="https://techwritingmatters.com/how-to-update-your-forked-repository-on-github">document</a> is helpful.  Be sure to add the professors and TAs as collaborators on your project. 

- lpalum@gmail.com GitHub ID: lpalum
- ajay.anand@rochester.edu GitHub ID: ajayan12
- divyamunot1999@gmail.com GitHub ID: divyamunot
- ylong6@u.Rochester.edu GitHub ID: NinaLong2077

Once you have updates your fork of the repository you should see the following template project that is resident in the final_project directory.
</p>

<img src="https://data-science-at-scale.s3.amazonaws.com/images/notebooks.drawio.png">

<p>
You can then pull your project into the Databrick Workspace using the <a href="https://www.databricks.training/step-by-step/importing-courseware-from-github/index.html">Repos</a> feature.
Each student is expected to submit the URL of their project on GitHub with their code checked in on the main/master branch.  This illustration highlights the branching scheme that you may use to work on your code in steps and then merge your submission into your master branch before submitting.
</p>
<img src="https://data-science-at-scale.s3.amazonaws.com/images/github.drawio.png">
<p>
Work your way through this notebook which will give you the steps required to submit a complete and compliant project.  The following illustration and associated data dictionary specifies the transformations and data that you are to generate for each step in the medallion pipeline.
</p>
<br><br>
<img src="https://data-science-at-scale.s3.amazonaws.com/images/dataframes.drawio.png">

#### Bronze Data - raw ingest
- date - string in the source json
- user - string in the source json
- text - tweet string in the source json
- sentiment - the given sentiment of the text as determined by an unknown model that is provided in the source json
- source_file - the path of the source json file the this row of data was read from
- processing_time - a timestamp of when you read this row from the source json

#### Silver Data - Bronze Preprocessing
- timestamp - convert date string in the bronze data to a timestamp
- mention - every @username mentioned in the text string in the bronze data gets a row in this silver data table.
- cleaned_text - the bronze text data with the mentions (@username) removed.
- sentiment - the given sentiment that was associated with the text in the bronze table.

#### Gold Data - Silver Table Inference
- timestamp - the timestamp from the silver data table rows
- mention - the mention from the silver data table rows
- cleaned_text - the cleaned_text from the silver data table rows
- sentiment - the given sentiment from the silver data table rows
- predicted_score - score out of 100 from the Hugging Face Sentiment Transformer
- predicted_sentiment - string representation of the sentiment
- sentiment_id - 0 for negative and 1 for postive associated with the given sentiment
- predicted_sentiment_id - 0 for negative and 1 for positive assocaited with the Hugging Face Sentiment Transformer

#### Application Data - Gold Table Aggregation
- min_timestamp - the oldest timestamp on a given mention (@username)
- max_timestamp - the newest timestamp on a given mention (@username)
- mention - the user (@username) that this row pertains to.
- negative - total negative tweets directed at this mention (@username)
- neutral - total neutral tweets directed at this mention (@username)
- positive - total positive tweets directed at this mention (@username)

When you are designing your approach, one of the main decisions that you will need to make is how you are going to orchestrate the streaming data processing in your pipeline.  There are several valid approaches.  First, you may choose to start the bronze_stream and let it complete (read and append all of the source data) before preceeding and starting up the silver_stream.  This approach has latency associated with it but it will allow your code to proceed in a linear fashion and process all the data by the end of your notebook execution.  Another potential approach is to start all the streams and have a "watch" method to determine when the pipeline has processed sufficient or all of the source data before stopping and displaying results.  Both of these approaches are valid and have different implications on how you will trigger your steams and how you will gate the execution of your pipeline.  Think through how you want to proceed and ask questions if you need guidance. The following references may be helpful:
- [Spark Structured Streaming Programming Guide](https://spark.apache.org/docs/latest/structured-streaming-programming-guide.html)
- [Databricks Autoloader - Cloudfiles](https://docs.databricks.com/en/ingestion/auto-loader/index.html)

### Be sure that your project runs end to end when *Run all* is executued on this notebook! (15 Points out of 60)

In [0]:
%run ./includes/includes

In [0]:
"""
Adding a widget to the notebook to control the clearing of a previous run.
or stopping the active streams using routines defined in the utilities notebook
"""
dbutils.widgets.removeAll()

dbutils.widgets.dropdown("clear_previous_run", "No", ["No","Yes"])
if (getArgument("clear_previous_run") == "Yes"):
    clear_previous_run()
    print("Cleared all previous data.")

dbutils.widgets.dropdown("stop_streams", "No", ["No","Yes"])
if (getArgument("stop_streams") == "Yes"):
    stop_all_streams()
    print("Stopped all active streams.")

from delta import *
dbutils.widgets.dropdown("optimize_tables", "No", ["No","Yes"])
if (getArgument("optimize_tables") == "Yes"):
    # Suck up those small files that we have been appending.
    DeltaTable.forPath(spark, BRONZE_DELTA).optimize().executeCompaction()
    # Suck up those small files that we have been appending.
    DeltaTable.forPath(spark, SILVER_DELTA).optimize().executeCompaction()
    # Suck up those small files that we have been appending.
    DeltaTable.forPath(spark, GOLD_DELTA).optimize().executeCompaction()
    print("Optimized all of the Delta Tables")

## 1.0 Import your libraries here...
- Are your shuffle partitions consistent with your cluster and your workload?
- Do you have the necessary libraries to perform the required operations in the pipeline/application?

In [0]:
from pyspark.sql.types import (
    StructType, 
    StructField, 
    StringType, 
    TimestampType
)
from pyspark.sql.functions import (
    input_file_name, 
    current_timestamp, 
    to_timestamp,
    regexp_replace,
    split,
    length,
    explode,
    isnan, 
    isnull, 
    col,
    min,
    max,
    sum,
    desc,
    lit,
    when
)
from delta.tables import DeltaTable

import matplotlib.pyplot as plt

import mlflow
from mlflow.pyfunc import spark_udf
from sklearn.metrics import (
    precision_score, 
    recall_score, 
    f1_score, 
    confusion_matrix,
    ConfusionMatrixDisplay,
)

## 2.0 Use the utility functions to ...
- Read the source file directory listing
- Count the source files (how many are there?)
- print the contents of one of the files

In [0]:
# Read source file directory
files = get_source_listing_df()

# Count source files
print(f"There are {len(files)} files in the source directory.")

# Print first file
FIRST_FILE = "voc_volume/0.json"
print("The first file looks like:")
print(show_s3_file_contents(FIRST_FILE))

## 3.0 Transform the Raw Data to Bronze Data using a stream
- define the schema for the raw data
- setup a read stream using cloudfiles and the source data format
- setup a write stream using cloudfiles to append to the bronze delta table
- enforce schema
- allow a new schema to be merged into the bronze delta table
- Use the defined BRONZE_CHECKPOINT and BRONZE_DELTA paths defines in the includes
- name your raw to bronze stream as bronze_stream
- transform the raw data to the bronze data using the data definition at the top of the notebook

In [0]:
# Define the schema based on the raw tweets
schema = StructType([
    StructField("date", StringType()),
    StructField("user", StringType()),
    StructField("text", StringType()),
    StructField("sentiment", StringType()),
])

In [0]:
# Read stream setup
read_stream = (spark.readStream
    .format("cloudFiles")
    .option("cloudFiles.format", "json")
    .option("cloudFiles.includeExistingFiles", "true")
    .schema(schema)
    .load(TWEET_SOURCE_PATH)
)

# Bronze write stream
bronze_stream = (read_stream
    .withColumn("source_file", input_file_name())
    .withColumn("processing_time", current_timestamp())
    .writeStream
    .queryName("raw_to_bronze_stream")
    .format("delta")
    .outputMode("append")
    .option("checkpointLocation", BRONZE_CHECKPOINT)
    .option("path", BRONZE_DELTA)
    .start()
)

In [0]:
# Peek and store bronze table
bronze = spark.read.format("delta").load(BRONZE_DELTA)
bronze.display()

## 4.0 Bronze Data Exploratory Data Analysis
- How many tweets are captured in your Bronze Table?
- Are there any columns that contain Nan or Null values?  If so how many and what will you do in your silver transforms to address this?
- Count the number of tweets by each unique user handle and sort the data by descending count.
- How many tweets have at least one mention (@) how many tweet have no mentions (@)
- Plot a bar chart that shows the top 20 tweeters (users)


In [0]:
# Count tweets
tweet_count = bronze.count()
print(f"{tweet_count} tweets in Bronze Delta Table")

In [0]:
# Check for null or NaN
null_columns = []
for c in bronze.columns:
    if bronze.filter(col(c).isNull()).count() > 0:
        null_columns.append(c)
        
print("The columns that contain null or NaN values:", null_columns)

In [0]:
# Tweets per user, sorted in descending
tweet_count_by_user = bronze.groupBy("user").count().orderBy("count", ascending=False)
display(tweet_count_by_user.head(10))

In [0]:
# Count the number of tweets with at least one mention
mention_count = bronze.filter(col("text").like("%@%")).count()

no_mention_count = tweet_count - mention_count

# Print the results
print("Number of tweets with at least one mention: ", mention_count)
print("Number of tweets with no mentions: ", no_mention_count)

In [0]:
top_20_users = tweet_count_by_user.limit(20).toPandas()

plt.bar(top_20_users['user'], top_20_users['count'])
plt.xticks(rotation=45, ha="right")
plt.xlabel('User')
plt.ylabel('Number of Tweets')
plt.title('Top 20 Tweeters')
plt.show()

## 5.0 Transform the Bronze Data to Silver Data using a stream
- setup a read stream on your bronze delta table
- setup a write stream to append to the silver delta table
- Use the defined SILVER_CHECKPOINT and SILVER_DELTA paths in the includes
- name your bronze to silver stream as silver_stream
- transform the bronze data to the silver data using the data definition at the top of the notebook

In [0]:
# Set time parser policy to LEGACY
spark.conf.set("spark.sql.legacy.timeParserPolicy", "LEGACY")

# Read stream setup
read_stream = (spark.readStream
    .format("delta")
    .load(BRONZE_DELTA)
)

# Silver write stream
silver_stream = (read_stream
    # formats like Fri May 01 21:30:59 PDT 2009
    .select(to_timestamp(col("date"), "EEE MMM dd HH:mm:ss zzz yyyy").alias("timestamp"), "text", "sentiment")
    .withColumn("mention", explode(split(col("text"), " ")))
    .filter(col("mention").startswith("@"))
    .filter(length(col("mention")) > 1)
    .withColumn("cleaned_text", regexp_replace(col("text"), "@[a-zA-Z0-9_]+", ""))
    .select("timestamp", "mention", "cleaned_text", "sentiment")
    .writeStream
    .queryName("bronze_to_silver_stream")
    .format("delta")
    .outputMode("append")
    .option("checkpointLocation", SILVER_CHECKPOINT)
    .option("path", SILVER_DELTA)
    .start()
)

In [0]:
# Peek at the silver data
silver = spark.read.format("delta").load(SILVER_DELTA)
silver.display()

## 6.0 Transform the Silver Data to Gold Data using a stream
- setup a read stream on your silver delta table
- setup a write stream to append to the gold delta table
- Use the defined GOLD_CHECKPOINT and GOLD_DELTA paths defines in the includes
- name your silver to gold stream as gold_stream
- transform the silver data to the gold data using the data definition at the top of the notebook
- Load the pretrained transformer sentiment classifier from the MODEL_NAME at the production level from the MLflow registry
- Use a spark UDF to parallelize the inference across your silver data

In [0]:
MODEL_URI = f"models:/{MODEL_NAME}/production"

# Load model
model_udf = spark_udf(spark, MODEL_URI)

In [0]:
# Read stream setup
read_stream = (spark.readStream
    .format("delta")
    .load(SILVER_DELTA)
)
# Gold write stream
gold_stream = (read_stream
    .withColumn("predicted", model_udf("cleaned_text"))
    .select("timestamp", "mention", "cleaned_text", "sentiment", col("predicted.score").alias("predicted_score"), col("predicted.label").alias("predicted_sentiment"))
    .drop("predicted")
    .withColumn("sentiment_id", 
                when(col("sentiment") == "positive", 1)
                .otherwise(-1)
                )
    .withColumn("predicted_sentiment_id", 
                when(col("predicted_sentiment") == "POS", 1)
                .when(col("predicted_sentiment") == "NEU", 0)
                .otherwise(-1)
                )
    .writeStream
    .queryName("silver_to_gold_stream")
    .format("delta")
    .outputMode("append")
    .option("checkpointLocation", GOLD_CHECKPOINT)
    .option("path", GOLD_DELTA)
    .start()
)

In [0]:
# Peek at the gold table
gold = spark.read.format("delta").load(GOLD_DELTA)
gold.display()

## 7.0 Capture the accuracy metrics from the gold table in MLflow
Store the following in an MLflow experiment run:
- Store the precision, recall, and F1-score as MLflow metrics
- Store an image of the confusion matrix as an MLflow artifact
- Store the mdoel name and the MLflow version that was used as an MLflow parameters
- Store the version of the Delta Table (input-silver) as an MLflow parameter

### Implementation details

Since our model gives us not a binary but a 3-class output (POS, NEU, NEG), we try two ways of dealing with the Neutral category. We display the confusion matrix with the Neutral class (there are 0 Neutral class in the true labels) for posterity. Then we store two runs, one where the Neutral labels are treated as Negative, and one where the Neutral labels are treated as Positive. We see better results treating the Neutral labels as Positive.

In [0]:
# Get version number
deltaTable = DeltaTable.forPath(spark, SILVER_DELTA)
version_num = deltaTable.history().toPandas()["version"].iloc[-1]

# Filter dataframe by predicted_sentiment_id != 0
neutral_as_positive = gold.withColumn("predicted_sentiment_id", when(col("predicted_sentiment_id") == 0, 1).otherwise(col("predicted_sentiment_id")))
neutral_as_negative = gold.withColumn("predicted_sentiment_id", when(col("predicted_sentiment_id") == 0, -1).otherwise(col("predicted_sentiment_id")))

# Generate confusion matrix with all classes
predicted_labels = gold.select("predicted_sentiment_id").collect()
true_labels = gold.select("sentiment_id").collect()

disp = ConfusionMatrixDisplay(confusion_matrix=confusion_matrix(true_labels, predicted_labels), display_labels=["Negative", "Neutral", "Positive"])
disp.plot()

# Calculate the stats for both:
for exp_name, df in [("neutral -> positive", neutral_as_positive), ("neutral -> negative", neutral_as_negative)]:
    # Get labels
    predicted_labels = df.select("predicted_sentiment_id").collect()
    true_labels = df.select("sentiment_id").collect()

    # Calculate metrics
    precision = precision_score(true_labels, predicted_labels)
    recall = recall_score(true_labels, predicted_labels)
    f1 = f1_score(true_labels, predicted_labels)

    # Generate confusion matrix
    dbutils.fs.mkdirs("dbfs:/FileStore/figs/")
    cm_image_path = "/dbfs/FileStore/figs/confusion_matrix.png"
    disp = ConfusionMatrixDisplay(confusion_matrix=confusion_matrix(true_labels, predicted_labels), display_labels=["Negative", "Positive"])
    disp.plot()
    disp.figure_.savefig(cm_image_path)

    # Log metrics
    with mlflow.start_run(run_name=exp_name):
        mlflow.log_metric("precision", precision)
        mlflow.log_metric("recall", recall)
        mlflow.log_metric("f1_score", f1)

        mlflow.log_artifact(cm_image_path)

        mlflow.log_param("model_name", MODEL_NAME)
        mlflow.log_param("mlflow_version", mlflow.version.VERSION)

        # I also decided to log the URI of the model
        mlflow.log_param("model_uri", MODEL_URI)

        mlflow.log_param("silver_version", version_num)

## 8.0 Application Data Processing and Visualization
- How many mentions are there in the gold data total?
- Count the number of neutral, positive and negative tweets for each mention in new columns
- Capture the total for each mention in a new column
- Sort the mention count totals in descending order
- Plot a bar chart of the top 20 mentions with positive sentiment (the people who are in favor)
- Plot a bar chart of the top 20 mentions with negative sentiment (the people who are the vilians)

You may want to use the "Loop Application" widget to control whether you repeateded display the latest plots while the data comes in from your streams before moving on to the next section and cleaning up your run.

*note: A mention is a specific twitter user that has been "mentioned" in a tweet with an @user reference.

In [0]:
# Batch create application data
application_df = (gold
                  .groupBy("mention")
                  .agg(
                      sum(when(col("predicted_sentiment") == "NEG", 1).otherwise(0)).alias("negative"),
                      sum(when(col("predicted_sentiment") == "NEU", 1).otherwise(0)).alias("neutral"),
                      sum(when(col("predicted_sentiment") == "POS", 1).otherwise(0)).alias("positive"),
                      sum(lit(1)).alias("total")
                      )
                  .orderBy(desc("total"))
)

application_df.display()

In [0]:
# Top 20 mentions with positive sentiment
top_positive_mentions = application_df.orderBy(desc("positive")).limit(20).toPandas()
plt.figure(figsize=(10, 6))
plt.bar(top_positive_mentions["mention"], top_positive_mentions["positive"])
plt.xlabel("Mentions")
plt.ylabel("Count")
plt.title("Top 20 Positive Mentions")
plt.xticks(rotation=45, ha="right")
plt.show()

# Top 20 mentions with negative sentiment
top_negative_mentions = application_df.orderBy(desc("negative")).limit(20).toPandas()
plt.figure(figsize=(10, 6))
plt.bar(top_negative_mentions["mention"], top_negative_mentions["negative"])
plt.xlabel("Mentions")
plt.ylabel("Count")
plt.title("Top 20 Negative Mentions")
plt.xticks(rotation=45, ha="right")
plt.show()

## 9.0 Clean up and completion of your pipeline
- using the utilities what streams are running? If any.
- Stop all active streams
- print out the elapsed time of your notebook.

In [0]:
# Print active streams
for stream in spark.streams.active:
    print(f"Stopping stream {stream.name} (was active)")
    stream.stop()

In [0]:
# Get the notebooks ending time note START_TIME was established in the include file when the notebook started.
END_TIME = time.time()
print(f"Total notebook runtime: {END_TIME - START_TIME} seconds.")

## 10.0 How Optimized is your Spark Application (Grad Students Only)
Graduate students (registered for the DSCC-402 section of the course) are required to do this section.  This is a written analysis using the Spark UI (link to screen shots) that support your analysis of your pipelines execution and what is driving its performance.
Recall that Spark Optimization has 5 significant dimensions of considertation:
- Spill: write to executor disk due to lack of memory
- Skew: imbalance in partition size
- Shuffle: network io moving data between executors (wide transforms)
- Storage: inefficiency due to disk storage format (small files, location)
- Serialization: distribution of code segments across the cluster

Comment on each of the dimentions of performance and how your impelementation is or is not being affected.  Use specific information in the Spark UI to support your description.  

Note: you can take sreenshots of the Spark UI from your project runs in databricks and then link to those pictures by storing them as a publicly accessible file on your cloud drive (google, one drive, etc.)

References:
- [Spark UI Reference Reference](https://spark.apache.org/docs/latest/web-ui.html#web-ui)
- [Spark UI Simulator](https://www.databricks.training/spark-ui-simulator/index.html)

### ENTER YOUR MARKDOWN HERE